In [11]:
import numpy as np
import torch
from torchvision import transforms
import cv2
import matplotlib.pyplot as plt
import os
import pandas as pd
# import model.resnet_mutilabels as models
# from torchvision import models
# import models
from torchvision.models.swin_transformer import Swin_B_Weights, Swin_T_Weights, Swin_S_Weights
from data import RoadBreak, RoadAllData
import random
import models
from PIL import Image
from copy import copy

In [131]:
x = torch.rand((4,6))
model1_ori = torch.nn.Sequential(
    torch.nn.Linear(6, 2), 
    torch.nn.Linear(2, 2)
)
model2_ori = torch.nn.Sequential(
    torch.nn.Linear(6, 2), 
    torch.nn.Linear(2, 2)
)
y = torch.rand((1,6))

model1_ori(y), model2_ori(y)

(tensor([[0.0303, 0.3741]], grad_fn=<AddmmBackward0>),
 tensor([[ 0.2369, -0.1491]], grad_fn=<AddmmBackward0>))

In [138]:
ce = torch.nn.CrossEntropyLoss()
target = torch.Tensor([0, 1, 1, 0]).type(torch.long)
model1 = torch.nn.Sequential(
    torch.nn.Linear(6, 2), 
    torch.nn.Linear(2, 2)
)
model2 = torch.nn.Sequential(
    torch.nn.Linear(6, 2), 
    torch.nn.Linear(2, 2)
)
for param, param_bak in zip(model1.parameters(), model1_ori.parameters()):
    param.data = param_bak.data.clone()

for param, param_bak in zip(model2.parameters(), model2_ori.parameters()):
    param.data = param_bak.data.clone()

optimizer = torch.optim.Adam(lr=1e-4, params=list(model1.parameters()) + list(model2.parameters()))
score = model1(x) + model2(x)
loss = ce(score, target)
optimizer.zero_grad()
loss.backward()
optimizer.step()
model1(y), model2(y)

(tensor([[0.0302, 0.3743]], grad_fn=<AddmmBackward0>),
 tensor([[ 0.2368, -0.1490]], grad_fn=<AddmmBackward0>))

In [135]:
ce = torch.nn.CrossEntropyLoss()
target = torch.Tensor([0, 1, 1, 0]).type(torch.long)
model1 = torch.nn.Sequential(
    torch.nn.Linear(6, 2), 
    torch.nn.Linear(2, 2)
)
model2 = torch.nn.Sequential(
    torch.nn.Linear(6, 2), 
    torch.nn.Linear(2, 2)
)
for param, param_bak in zip(model1.parameters(), model1_ori.parameters()):
    param.data = param_bak.data.clone()

for param, param_bak in zip(model2.parameters(), model2_ori.parameters()):
    param.data = param_bak.data.clone()
optimizer1 = torch.optim.Adam(lr=1e-4, params=model1.parameters())
optimizer2 = torch.optim.Adam(lr=1e-4, params=model2.parameters())
score = model1(x) + model2(x)
loss = ce(score, target)
optimizer1.zero_grad()
optimizer2.zero_grad()
loss.backward()
optimizer1.step()
optimizer2.step()
model1(y), model2(y)

(tensor([[0.0302, 0.3743]], grad_fn=<AddmmBackward0>),
 tensor([[ 0.2368, -0.1490]], grad_fn=<AddmmBackward0>))

In [2]:
hist_mean = np.load('data/hist_mean.npy')

In [4]:
hist_mean.shape

(4096,)

In [5]:
hist_std = np.load('data/hist_std.npy')

In [11]:
(hist_std == 0).sum()

2236

In [9]:
a = np.ones((2, 4096))
a[:, hist_mean==0].shape

(2, 2236)

In [ ]:
unique = np.zeros((0, 3)).astype('uint8')
for i in range(29100, 29160):
    print(i)
    img = cv2.imread(r'/data2/zrliu/dataset_cmp/dataset/train_image/labeled_data/train_{}.png'.format(i))
    unique = np.unique(np.concatenate([unique, np.unique(img.reshape(-1, 3), axis=0)]), axis=0)
print(len(unique))

In [28]:
img = cv2.imread(r'/data2/zrliu/dataset_cmp/dataset/train_image/labeled_data/train_29100.png'.format(i))

In [56]:
class Histogram(torch.nn.Module):
    def __init__(self, levels=16):
        self.levels = levels
        self.max_v = levels ** 3
        self.v_num = (256 // levels)
    def __call__(self, img):
        img = np.array(img)
        img = img // self.v_num
        img = img.astype(int)
        values = img[:, :, 2] * (self.levels * self.levels)
        values += img[:, :, 1] * self.levels
        values += img[:, :, 0]
        values = values.reshape(-1)
        values = np.insert(values, len(values), self.max_v)
        return np.bincount(values)

In [57]:
hist = Histogram(64)

In [59]:
data_path = '/data2/zrliu/dataset_cmp/dataset/train_image/labeled_data'
hists = []
print(len(os.listdir(data_path)))
for i, name in enumerate(os.listdir(data_path)):
    img = Image.open(os.path.join(data_path, name)).convert('RGB')
    img = np.array(img)
    
    hists.append(hist(img))
    if i % 1000 == 0:
        print(i)
        
        
hists = np.array(hists)
mean = hists.mean(axis=0)
std = hists.std(axis=0)

29164
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000


In [61]:
std[std>0].shape

(53155,)

In [64]:
mean[mean>0].shape

(53156,)

In [62]:
np.save('/home/zrliu/repo_semi_supervised/hist_mean_64.npy', mean)

In [70]:
mean[mean>0]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [63]:
np.save('/home/zrliu/repo_semi_supervised/hist_std_64.npy', std)

In [84]:
len(std[std == 0])

2236

In [81]:
hists.shape

(29164, 4096)

In [28]:
np.zeros((0,3))

array([], shape=(0, 3), dtype=float64)

In [9]:
from tabnanny import check


model = models.__dict__['resnet34']()
checkpoint = torch.load('/data2/zrliu/checkpoints/hwcmp/checkpoint_0009.pth.tar')
state_dict = checkpoint['state_dict']
for k in list(state_dict.keys()):
    # retain only encoder_q up to before the embedding layer
    if k.startswith('module.encoder_q') and not k.startswith('module.encoder_q.fc'):
        # remove prefix
        state_dict[k[len("module.encoder_q."):]] = state_dict[k]
    # delete renamed or unused k
    del state_dict[k]
msg = model.load_state_dict(state_dict, strict=False)

In [10]:
msg

_IncompatibleKeys(missing_keys=['fc.0.weight', 'fc.0.bias', 'fc.1.weight', 'fc.1.bias', 'fc.2.weight', 'fc.2.bias', 'fc.3.weight', 'fc.3.bias', 'fc.4.weight', 'fc.4.bias', 'fc.5.weight', 'fc.5.bias', 'fc.6.weight', 'fc.6.bias', 'fc.7.weight', 'fc.7.bias'], unexpected_keys=[])

: 

In [11]:
dataset = RoadAllData('/data2/zrliu/dataset_cmp/dataset', transform=None, mode='train', expand=False, split=args.split)

TypeError: RoadAllData.__init__() got an unexpected keyword argument 'rate_for_crop'

In [9]:
img = (np.random.rand(224, 224, 3) * 255).astype('uint8')
rb = RoadBreak()
rb(img)

ValueError: empty range for randrange() (0, 0, 0)

In [55]:
img1 = torch.rand((1,3,468,224)).cuda()
img2 = torch.rand((1,3,224,468)).cuda()
label = torch.randint(0, 1000, (1,)).cuda()
model = models.resnet34().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [69]:


out1 = model(img1)
# out2 = model(img2)
ce = torch.nn.CrossEntropyLoss().cuda()

loss = 0
loss += ce(out1, label)
# loss += ce(out2, label)
optimizer.zero_grad()
loss.backward()
model.fc.weight.grad

tensor([[0.0005, 0.0005, 0.0005,  ..., 0.0005, 0.0005, 0.0005],
        [0.0004, 0.0005, 0.0005,  ..., 0.0005, 0.0005, 0.0005],
        [0.0003, 0.0003, 0.0003,  ..., 0.0003, 0.0003, 0.0003],
        ...,
        [0.0048, 0.0049, 0.0052,  ..., 0.0050, 0.0052, 0.0052],
        [0.0006, 0.0006, 0.0006,  ..., 0.0006, 0.0006, 0.0006],
        [0.0007, 0.0008, 0.0008,  ..., 0.0008, 0.0008, 0.0008]],
       device='cuda:0')

(torch.Size([32, 512]), torch.Size([32, 512]))

(torch.Size([32, 512]), torch.Size([32, 512]))

In [48]:

# optimizer.step()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

torch.Size([1, 1000])

In [2]:
model = models.__dict__['swin_b'](pretrained=True)

In [3]:
model = models.__dict__['resnet18'](auxiliary=True)

In [23]:
model = models.swin_b(weights=Swin_B_Weights.IMAGENET1K_V1)
model.head = torch.nn.Sequential()

Downloading: "https://download.pytorch.org/models/swin_b-68c6b09e.pth" to /home/zrliu/.cache/torch/hub/checkpoints/swin_b-68c6b09e.pth


  0%|          | 0.00/335M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [16]:
img = torch.rand((1,3,498,224))
out = model(img)

In [17]:
out.shape

torch.Size([1, 768])

In [2]:
checkpoint = torch.load('/home/zrliu/epoch_30.pkl')
state_dict = checkpoint
for k in list(state_dict.keys()):
    state_dict[k[len("module."):]] = state_dict[k]
    # delete renamed or unused k
    del state_dict[k]
for k in list(state_dict.keys()):
    if k.startswith('backbone.'):
        state_dict[k[len("backbone."):]] = state_dict[k]
        # delete renamed or unused k
        del state_dict[k]

model = models.__dict__['resnet18']()
model.load_state_dict(state_dict)

<All keys matched successfully>

In [3]:
model2 = basemodel.Model()
checkpoint = torch.load('/home/zrliu/epoch_30.pkl')
state_dict = checkpoint
for k in list(state_dict.keys()):
    state_dict[k[len("module."):]] = state_dict[k]
    # delete renamed or unused k
    del state_dict[k]
model2.load_state_dict(state_dict)

/home/zrliu/anaconda3/envs/omnivore/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/zrliu/anaconda3/envs/omnivore/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [13]:
img = torch.rand((1,3,224,224))
out1, feats = model(img)
out2 = model2(img)[0]

In [17]:
for i in model2._modules.items():
    print(i)

('backbone', ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inp

In [5]:
out1

tensor([[ 6.0896, -6.0345]], grad_fn=<AddmmBackward0>)

In [6]:
out2

tensor([[ 6.0896, -6.0345]], grad_fn=<AddmmBackward0>)

In [2]:
from mydataset import RoadAllData

In [3]:
dataset_train = RoadAllData(dataset_path='/data2/zrliu/dataset_cmp/dataset', transform=None, cat='train')
dataset_val = RoadAllData(dataset_path='/data2/zrliu/dataset_cmp/dataset', transform=None, cat='val')

In [4]:
len(dataset_train), len(dataset_val)

(20380, 8784)

In [13]:
train_cls_dict, val_cls_dict = {}, {}

for i in range(dataset_train.label_list.shape[0]):
    cls = dataset_train.label_list[i].argmax().item()
    if cls not in train_cls_dict:
        train_cls_dict[cls] = 1
    else:
        train_cls_dict[cls] += 1

for i in range(dataset_val.label_list.shape[0]):
    cls = dataset_val.label_list[i].argmax().item()
    if cls not in val_cls_dict:
        val_cls_dict[cls] = 1
    else:
        val_cls_dict[cls] += 1

In [20]:
for i in range(8):
    print(i, '\t', train_cls_dict[i], '\t', round(train_cls_dict[i] / len(dataset_train), 4))

0 	 17981 	 0.8823
1 	 142 	 0.007
2 	 459 	 0.0225
3 	 164 	 0.008
4 	 158 	 0.0078
5 	 175 	 0.0086
6 	 1131 	 0.0555
7 	 170 	 0.0083


In [25]:
for i in range(8):
    print(i, '\t', train_cls_dict[i] + val_cls_dict[i], '\t', round((train_cls_dict[i] + val_cls_dict[i]) / (len(dataset_train) + len(dataset_val)), 4))

0 	 25767 	 0.8835
1 	 187 	 0.0064
2 	 638 	 0.0219
3 	 217 	 0.0074
4 	 234 	 0.008
5 	 255 	 0.0087
6 	 1624 	 0.0557
7 	 242 	 0.0083


In [48]:
data_path = '/data2/zrliu/dataset_cmp/dataset/train_label/train.csv'
new_data_path = '/data2/zrliu/dataset_cmp/dataset/train_label/train_expand.csv'
img_list, label_list = [], []
repeat_count = {'0':1, '1':120, '2':28, '3':105, '4':80, '5':90, '6':8, '7':85}
with open(data_path, 'r') as f:
    for line in f:
        line_split = line.replace('\n', '').split(',')
        rc = 0
        labels = []
        for label in line_split[1:]:
            if label != '':
                labels.append(label)
        for label in labels:
            rc += repeat_count[label]
        for i in range(rc): 
            img_list.append(line_split[0])
            label_list.append(labels)

new_count = {'0':0, '1':0, '2':0, '3':0, '4':0, '5':0, '6':0, '7':0}
for i in range(len(img_list)):
    for label in label_list[i]:
        new_count[label] += 1

for i in range(8):
    print(new_count[str(i)])




csv_str = ""
for i in range(len(img_list)):
    line = [img_list[i]] + label_list[i]
    csv_str += (','.join(line) + '\n')

with open(new_data_path, 'w') as f:
    f.write(csv_str)

17981
18002
17212
17968
17583
18092
17819
17868


In [52]:
new_data_path = '/data2/zrliu/dataset_cmp/dataset/train_label/train_expand.csv'
new_count = {'0':0, '1':0, '2':0, '3':0, '4':0, '5':0, '6':0, '7':0}
with open(new_data_path, 'r') as f:
    for line in f:
        split_ = line.replace('\n', '').split(',')
        for label in split_[1:]:
            new_count[label] += 1

for i in range(8):
    print(new_count[str(i)])

17981
18002
17212
17968
17583
18092
17819
17868
